# 初始化

In [1]:
import os

API_KEY = os.getenv('UNION_API_KEY')
BASE_URL = os.getenv('UNION_BASE_URL')

# 基本调用

In [2]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI()
llm.invoke("请介绍下你自己")

AIMessage(content='我是一个AI助手，专门为用户提供各种服务和帮助。我具有多种功能，包括回答问题、提供建议、进行翻译、解决问题等等。我可以通过文字、语音等方式与用户进行交流，并尽力满足用户的需求。我会不断学习和进化，以提供更好的服务。', response_metadata={'token_usage': {'completion_tokens': 104, 'prompt_tokens': 16, 'total_tokens': 120}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-eb5128b4-3443-4161-afe9-1e467e81d2b8-0')

# 使用模板

In [3]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are world class technical documentation writer."),
    ("user", "{input}")
])

print(prompt)

# 模板输出作为llm输入
chain = prompt | llm
chain.invoke({"input": "武汉在哪里？"})

input_variables=['input'] messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are world class technical documentation writer.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))]


AIMessage(content='武汉是中国的一个城市，位于中国中部，长江和汉江的交汇处。它是湖北省的省会，也是中国重要的中心城市之一。武汉地处中国的内陆地区，与上海、北京等沿海城市相距较远。它是中国最大的内陆港口城市之一，也是重要的交通枢纽。', response_metadata={'token_usage': {'completion_tokens': 114, 'prompt_tokens': 28, 'total_tokens': 142}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-5ec06b72-3cb2-4877-b21e-2e915f7cb7f2-0')

# 添加解析器，将大模型的输出转换成普通字符串

In [4]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()
chain = prompt | llm | output_parser
chain.invoke({"input": "武汉在哪里？"})

'武汉位于中国中部，是湖北省的省会城市。它位于长江和汉江的交汇处，被誉为"九省通衢"。武汉是中国重要的经济、交通和文化中心之一，也是中国内地最大的城市之一。'

# 检索链`Retrieval Chain`

In [5]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain

import os

os.environ['http_proxy'] = 'http://127.0.0.1:10801'
os.environ['http_proxy'] = 'https://127.0.0.1:10801'

loader = WebBaseLoader("https://docs.smith.langchain.com/user_guide")

llm = ChatOpenAI(api_key=API_KEY, model_name="gpt-3.5-turbo", base_url=BASE_URL)

# llm.invoke("武汉在哪里？")
# 加载文档
docs = loader.load()

# # 创建Embedding对象
embeddings = OpenAIEmbeddings()

# # 创建文本分割器
text_splitter = RecursiveCharacterTextSplitter()

# # 分割文本
documents = text_splitter.split_documents(docs)

# # 创建向量存储对象
vector = FAISS.from_documents(documents, embeddings)

prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

# # 创建一个文本链，将文档传递给大模型
document_chain = create_stuff_documents_chain(llm, prompt)

# # 创建一个检索链
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)
response = retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
print(response["answer"])

LangSmith can help with testing by allowing developers to create datasets, run tests on LLM applications, upload test cases in bulk or create them on the fly, export test cases from application traces, and run custom evaluations (both LLM and heuristic based) to score test results. Additionally, LangSmith provides a comparison view for tracking and diagnosing regressions in test scores across multiple revisions of an application. The platform also offers a playground environment for rapid iteration and experimentation, as well as the ability to collect data through beta testing to understand how LLM applications are performing in real-world scenarios. Users can capture feedback, annotate traces, add runs to datasets, and monitor key metrics over time for A/B testing and performance tracking. LangSmith also supports automations for processing traces in near real-time, making it easier to test and improve application performance.


# 对话检索链

In [35]:
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

# 创建一个包含历史消息的prompt，并要求大模型基于历史消息生成检索查询
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])

# 创建一个历史消息检索器
llm = ChatOpenAI(api_key=API_KEY, model_name="gpt-3.5-turbo", base_url=BASE_URL)
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

# 构建一个历史消息列表
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
print(chat_history)

# 传入历史消息和用户输入的问题，调用检索链
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[HumanMessage(content='Can LangSmith help test my LLM applications?'), AIMessage(content='Yes!')]


[Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug where it is failing — is incredibly important for this phase.Debugging\u200bWhen developing new LLM applications, we suggest having LangSmith tracing enabled

In [37]:
# 创建一个文档 chain，要求大模型基于上下文回答问题，同时嵌入聊天历史
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

# 创建一个检索链，将文档链和检索器串联起来
retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

# 构建一个历史消息列表，将检索链和聊天历史串联起来
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='Skip to main contentLangSmith API DocsSearchGo to AppQuick StartUser GuideTracingEvaluationProduction Monitoring & AutomationsPrompt HubProxyPricingSelf-HostingCookbookUser GuideOn this pageLangSmith User GuideLangSmith is a platform for LLM application development, monitoring, and testing. In this guide, we’ll highlight the breadth of workflows LangSmith supports and how they fit into each stage of the application development lifecycle. We hope this will inform users how to best utilize this powerful platform or give them something to consider if they’re just starting their journey.Prototyping\u200bPrototyping LLM applications often involves quick experimentation between prompts, model types, retrieval strategy and other parameters.\nThe ability to rapidly understand how the model is performing — and debug wh

# Agent

In [39]:
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_openai import ChatOpenAI
from langchain import hub
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

# 定义一个搜索工具
search = TavilySearchResults()

# 定义一个 Retriever Tool
retriever_tool = create_retriever_tool(
    retriever,
    "langsmith_search",
    "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!",
)

# 构建工具集
tools = [retriever_tool, search]

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

# You need to set OPENAI_API_KEY environment variable or pass it as argument `api_key`.
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
agent = create_openai_functions_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "what is the weather in SF?"})



> Entering new AgentExecutor chain...

Invoking: `tavily_search_results_json` with `{'query': 'weather in San Francisco'}`


[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.78, 'lon': -122.42, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1713181995, 'localtime': '2024-04-15 4:53'}, 'current': {'last_updated_epoch': 1713181500, 'last_updated': '2024-04-15 04:45', 'temp_c': 11.1, 'temp_f': 52.0, 'is_day': 0, 'condition': {'text': 'Overcast', 'icon': '//cdn.weatherapi.com/weather/64x64/night/122.png', 'code': 1009}, 'wind_mph': 4.3, 'wind_kph': 6.8, 'wind_degree': 280, 'wind_dir': 'W', 'pressure_mb': 1021.0, 'pressure_in': 30.14, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 86, 'cloud': 100, 'feelslike_c': 10.4, 'feelslike_f': 50.7, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 1.0, 'gust_mph': 7.4, 'gust_kph': 11.9}}"}, {'url': 'https://www.weathertab.com/en/c/e/04/uni

{'input': 'what is the weather in SF?',
 'output': 'The current weather in San Francisco is overcast with a temperature of 52.0°F (11.1°C). The wind is coming from the west at 4.3 mph (6.8 kph), and the humidity is 86%. The visibility is 9.0 miles (16.0 km), and there is no precipitation.'}

### 使用 retriver 作为工具

In [40]:
chat_history = [HumanMessage(content="Can LangSmith help test my LLM applications?"), AIMessage(content="Yes!")]
agent_executor.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})



> Entering new AgentExecutor chain...
LangSmith can help test your LLM applications in the following ways:

1. Personal Statement Review: LangSmith can provide feedback and suggestions on your personal statement to ensure it effectively communicates your motivations, experiences, and goals. This can help you make a strong impression on the admissions committee.

2. Resume/CV Review: LangSmith can review your resume or CV and provide recommendations on how to highlight your relevant experiences, skills, and achievements. This can help you present a compelling case for your admission.

3. Application Strategy: LangSmith can help you develop a strategic approach to your LLM applications, including selecting the right programs, organizing your application materials, and managing deadlines. This can help you maximize your chances of getting accepted to your desired programs.

4. Interview Preparation: LangSmith can assist you in preparing for LLM interviews by conducting mock interviews, 

{'chat_history': [HumanMessage(content='Can LangSmith help test my LLM applications?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'output': 'LangSmith can help test your LLM applications in the following ways:\n\n1. Personal Statement Review: LangSmith can provide feedback and suggestions on your personal statement to ensure it effectively communicates your motivations, experiences, and goals. This can help you make a strong impression on the admissions committee.\n\n2. Resume/CV Review: LangSmith can review your resume or CV and provide recommendations on how to highlight your relevant experiences, skills, and achievements. This can help you present a compelling case for your admission.\n\n3. Application Strategy: LangSmith can help you develop a strategic approach to your LLM applications, including selecting the right programs, organizing your application materials, and managing deadlines. This can help you maximize your chances of getting accepted to your desired prog